# Airtable Scraping

## Requirements

For scraping, I don't want to use unnecessary package, so basically the only requirement to be installed is `beautifulsoup4`, so just install it and you're ready to go!

In [72]:
!conda install beautifulsoup4 -q -y

#or using pip

#!pip install beautifulsoup4

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/banditelol/miniconda3

  added / updated specs:
    - beautifulsoup4
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py38_0         171 KB
    conda-4.8.4                |           py38_0         2.8 MB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.6.2~ --> pkgs/main::ca-certificates-2020.7.22-0

The following packages will be SUPERSEDED by a higher-priority channel:

  beautifulsoup4     conda-forge/noarch::beautifulsoup4-4.~ --> pkgs/main/linux-64::beautifulsoup4-4.9.1-py38_0
  certifi            conda-forge::certifi-2020.6.20-py38h3~ --> pkgs/main::certifi-2020.6.

## Package Used

We'll need the following package to be able to get csv data from airtable
- `requests` to make an HTTP requests
- `re` will be used to extract string using regular expression
- `bs4` is actually not a must need, but it feels ridiculous to work with html text without this package

In [69]:
import requests
import re
from bs4 import BeautifulSoup as bs

## Setup Some Variables

There are several constant that we can setup in the beginning and can be changed according to your needs. Make sure to change the `table` constant according to your shared table URL.

In [59]:
table = "https://airtable.com/shr5aMEsXCxORIwhk"
locale = "en"
time_zone = "Asia%2FJakarta"

html = bs(requests.get(table).text)

## Extract Necessary Parameters

As mentioned in my post, we need to extract the necessary parameters. But first, because all of the necessary information exists in the `<script>` tag after the `<title>` tag, thus we'll just extract the content

In [60]:
script = html.title.find_next('script')
print(script.prettify())

<script nonce="NxeRTfdEJIet">
 (function() {
if (window.fetch) {
var headers = {"x-user-locale":"*","x-airtable-application-id":"app7CUJtL7vQWbiDA","x-airtable-page-load-id":"pgls0eXzGWD71wQ5I","X-Requested-With":"XMLHttpRequest","x-airtable-inter-service-client":"webClient","ot-tracer-spanid":"32a9ea54746ed308","ot-tracer-traceid":"786fcdcf605dbd68","ot-tracer-sampled":"true"};
headers['x-time-zone'] = Intl.DateTimeFormat() ? Intl.DateTimeFormat().resolvedOptions().timeZone : 'UTC';
var timeoutPromise = new Promise(function(resolve, reject) {
setTimeout(
function() { reject('timeout'); },
30000);
});
window.__stashedPrefetch = {
metadata: {"method":"get","url":"\u002Fv0.3\u002Fview\u002FviwJuiikzJnQ0cyqj\u002FreadSharedViewData","objectParams":{"shouldUseNestedResponseFormat":true}},
earlyPrefetchSpan: {"ot-tracer-spanid":"32a9ea54746ed308","ot-tracer-traceid":"786fcdcf605dbd68","ot-tracer-sampled":"true"},
fetchStartTimeMs: Date.now(),
requestId: "req6GxdPUZVFYnHOk",
timeoutPromise: 

Now with the search scope narrowed we need to find the following informations:

- **ViewId** : a sequence of alphanumeric string starting with `viw`
- **AccessPolicy** : a URL encoded string preceded by `accessPolicy=`
- **AppId** : a sequence of alphanumeric string starting with `app` and preceded by `x-airtable-application-id` 

After we've found those parameters, we could now build the dictionary to help us in building the download requests later.

In [61]:
view_id = re.search(r"(viw[a-zA-Z0-9]+)",str(script)).group(1)
access_policy = re.search(r"accessPolicy=([a-zA-Z0-9%*]+)",str(script)).group(1)
app_id = re.search(r"\"x-airtable-application-id\":\"(app[a-zA-Z0-9]+)",str(script)).group(1)

params = {"x-time-zone":time_zone, "x-user-locale":locale, "x-airtable-application-id":app_id , "accessPolicy":access_policy}

## Build the Request URL


In [64]:
csv_url = f"https://airtable.com/v0.3/view/{view_id}/downloadCsv?"

for (k,v) in params.items():
    csv_url += k+"="+v+"&"
csv_url = csv_url[:-1]
r = requests.get(csv_url)

print(r.text)

﻿Story name,User want,Server points,Client points,Built?,Needs review,Facet,Epics,Blocks release,Sprint,Release seq. no.,Blocking stories,So that,FhabTask,P[?],Server eng. tasks,Client eng. tasks,Notes,FhabURL
S143: Improve adapter saltwater resistance,Improve adapter saltwater resistance,5,1,,checked,Adapter,E113: Amphibious vehicle support,007/in-beta partnership,Backlog — required,4,,"in case the adapter gets dropped in the ocean, it still works",13906,,,,,https://fhabricator.automatic.co/T13906
S133: Voiced alert when flux capacitor disconnected from adapter,Voiced alert when flux capacitor disconnected from adapter,5,8,checked,,Device connection,E110: Temporal displacement UX refinements,Beta1,Backlog — required,3,,fewer time travel accidents,12324,,,,,https://fhabricator.automatic.co/T12324
S138: Change progress bar graphics for giant robot transformation sequence,Change progress bar graphics for giant robot transformation sequence,2,3,checked,,Global,"E106: Full release style re

As you can see, the start of the csv there are `\xef\xbb\xbf` which are the [UTF8 encoded version of the unicode ZERO WIDTH NO-BREAK SPACE U+FEFF](https://stackoverflow.com/a/50131187), this can be resolved by changing the encoding of our response object to `utf-8`.

In [65]:
r.encoding = "utf-8"

print(r.text)

﻿Story name,User want,Server points,Client points,Built?,Needs review,Facet,Epics,Blocks release,Sprint,Release seq. no.,Blocking stories,So that,FhabTask,P[?],Server eng. tasks,Client eng. tasks,Notes,FhabURL
S143: Improve adapter saltwater resistance,Improve adapter saltwater resistance,5,1,,checked,Adapter,E113: Amphibious vehicle support,007/in-beta partnership,Backlog — required,4,,"in case the adapter gets dropped in the ocean, it still works",13906,,,,,https://fhabricator.automatic.co/T13906
S133: Voiced alert when flux capacitor disconnected from adapter,Voiced alert when flux capacitor disconnected from adapter,5,8,checked,,Device connection,E110: Temporal displacement UX refinements,Beta1,Backlog — required,3,,fewer time travel accidents,12324,,,,,https://fhabricator.automatic.co/T12324
S138: Change progress bar graphics for giant robot transformation sequence,Change progress bar graphics for giant robot transformation sequence,2,3,checked,,Global,"E106: Full release style re

Now that we have the csv data, we could save it to csv or pass it to other service, or host the code in google function to be used as backup service. Or just save it locally using the `file` package.